In [29]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV

In [30]:
newsgroups = fetch_20newsgroups(subset="all", categories=["alt.atheism", "sci.space"])
X = newsgroups.data
y = newsgroups.target

In [31]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)
X

<1786x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 303138 stored elements in Compressed Sparse Row format>

In [32]:
grid = {"C": np.power(10.0, np.arange(-5, 6))}
clf = SVC(kernel = 'linear',random_state = 241)
cv = KFold(shuffle = True,random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [33]:
C = gs.best_params_.get('C')
C

1.0

In [45]:
clf1 = SVC(kernel = 'linear',C = C,random_state = 241)
clf1.fit(X,y)

SVC(kernel='linear', random_state=241)

In [56]:
words = np.array(vectorizer.get_feature_names())
word_weights = pd.Series(clf1.coef_.data, index=words[clf1.coef_.indices], name="weight")
word_weights.index.name = "word"
top_words = word_weights.abs().sort_values(ascending=False).head(10)
top_words

word
space       2.663165
god         1.920379
atheism     1.254690
atheists    1.249180
moon        1.201611
sky         1.180132
religion    1.139081
bible       1.130612
keith       1.097094
sci         1.029307
Name: weight, dtype: float64

In [53]:
clf1.coef_.data

array([ 0.11331532,  0.05134321,  0.05445196, ..., -0.2372855 ,
        0.18461124, -0.2410018 ])

In [57]:
ans = " ".join(top_words.index.sort_values(ascending=True))
ans

'atheism atheists bible god keith moon religion sci sky space'